## More example applications

### Signal frontier analysis

In [9]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

start_dt=datetime.datetime(2009,1,1)
end_dt=datetime.datetime(2012,6,1)
from pandas_datareader import data as web
stock='AAPL'
px=pd.DataFrame({'AAPL':web.get_data_yahoo(stock, start_dt, end_dt)['Adj Close']})
names=['GOOG','MSFT', 'GS','INTC', 'MS', 'BAC', 'C']
for stock in names:
    while True:
        try:
            px[stock]=web.get_data_yahoo(stock, start_dt, end_dt)['Adj Close']
            break
        except:
            print('Unable to read stock: {0}'.format(stock))
            print('trying again')


In [10]:
#px = pd.read_csv('ch11/stock_px.csv')

In [11]:
plt.close('all')

In [12]:
px = px.asfreq('B').fillna(method='pad')
rets = px.pct_change()
((1 + rets).cumprod() - 1).plot()

In [13]:
def calc_mom(price, lookback, lag):
    mom_ret = price.shift(lag).pct_change(lookback)
    ranks = mom_ret.rank(axis=1, ascending=False)
    demeaned = ranks.subtract(ranks.mean(axis=1), axis=0)
    return demeaned.divide(demeaned.std(axis=1), axis=0)

In [14]:
compound = lambda x : (1 + x).prod() - 1
daily_sr = lambda x: x.mean() / x.std()

def strat_sr(prices, lb, hold):
    # Compute portfolio weights
    freq = '%dB' % hold
    port = calc_mom(prices, lb, lag=1)

    daily_rets = prices.pct_change()

    # Compute portfolio returns
    #port = port.shift(1).resample(freq, how='first')
    port = port.shift(1).resample(freq).first() #, how='first')
    returns = daily_rets.resample(freq).apply(compound)
    port_rets = (port * returns).sum(axis=1)

    return daily_sr(port_rets) * np.sqrt(252 / hold)

In [15]:
strat_sr(px, 70, 30)

0.35244218483851608

In [16]:
from collections import defaultdict

lookbacks = range(20, 90, 5)
holdings = range(20, 90, 5)
dd = defaultdict(dict)
for lb in lookbacks:
    for hold in holdings:
        dd[lb][hold] = strat_sr(px, lb, hold)

ddf = pd.DataFrame(dd)
ddf.index.name = 'Holding Period'
ddf.columns.name = 'Lookback Period'

In [17]:
ddf

Lookback Period,20,25,30,35,40,45,50,55,60,65,70,75,80,85
Holding Period,,,,,,,,,,,,,,
20,-0.506539,-0.635062,-0.693594,-0.609378,-0.078438,-0.636543,-0.015771,0.018017,0.055586,-0.093076,-0.091960,0.086202,0.076655,0.143128
25,-0.036261,0.060378,0.321525,0.379997,0.096943,0.087599,0.345740,0.420713,0.316453,0.338491,0.311224,-0.081075,-0.094650,-0.223975
30,-0.730025,-0.656360,-0.538480,-0.363146,-0.066197,-0.129353,0.110072,0.128934,0.233053,0.229879,0.352442,0.399556,0.186979,0.110632
35,-0.108609,0.103372,0.082990,0.650311,0.252828,0.308051,0.341147,0.476158,0.440549,0.378286,0.084437,0.003543,0.055474,0.104060
40,-0.243560,-0.290216,-0.422394,-0.404747,0.055972,-0.119520,-0.078666,0.093997,0.309298,0.144018,-0.025258,-0.056320,-0.038656,0.082771
45,0.353113,0.325406,0.339002,0.313961,0.366957,0.561694,0.474075,0.483056,0.481988,0.443656,0.484713,0.490460,0.465898,0.335682
50,0.357209,-0.283854,-0.012662,-0.079422,-0.183184,-0.213962,0.393442,0.607183,0.546785,0.516785,0.233110,0.432484,0.389677,0.368128
55,-0.669356,-0.583755,-0.534160,-0.155140,-0.376720,-0.282838,-0.214605,0.005517,-0.204439,-0.121295,0.094348,0.022490,0.091561,-0.102935
60,-0.450562,-0.604386,-0.591176,-0.495193,-0.230379,-0.291247,-0.296997,-0.294340,0.205928,0.277786,0.016349,-0.191714,0.053983,0.234133


In [18]:


def heatmap(df, cmap=plt.cm.gray_r):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    axim = ax.imshow(df.values, cmap=cmap, interpolation='nearest')
    ax.set_xlabel(df.columns.name)
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_xticklabels(list(df.columns))
    ax.set_ylabel(df.index.name)
    ax.set_yticks(np.arange(len(df.index)))
    ax.set_yticklabels(list(df.index))
    plt.colorbar(axim)

In [19]:
heatmap(ddf)
plt.show()

### Future contract rolling

In [20]:
pd.options.display.max_rows = 10

In [21]:
#import pandas.io.data as web
# Approximate price of S&P 500 index
px = web.get_data_yahoo('SPY')['Adj Close'] * 10
px

Date
2010-01-04     955.281067
2010-01-05     957.809753
2010-01-06     958.483963
2010-01-07     962.530136
2010-01-08     965.733109
                 ...     
2018-09-11    2890.499878
2018-09-12    2891.199951
2018-09-13    2908.299866
2018-09-14    2908.800049
2018-09-17    2893.399963
Name: Adj Close, Length: 2192, dtype: float64

In [22]:
from datetime import datetime
expiry = {'ESU2': datetime(2012, 9, 21),
          'ESZ2': datetime(2012, 12, 21)}
expiry = pd.Series(expiry).sort_values()

In [23]:
expiry

ESU2   2012-09-21
ESZ2   2012-12-21
dtype: datetime64[ns]

In [24]:
np.random.seed(12347)
N = 200
walk = (np.random.randint(0, 200, size=N) - 100) * 0.25
perturb = (np.random.randint(0, 20, size=N) - 10) * 0.25
walk = walk.cumsum()

rng = pd.date_range(px.index[0], periods=len(px) + N, freq='B')
near = np.concatenate([px.values, px.values[-1] + walk])
far = np.concatenate([px.values, px.values[-1] + walk + perturb])
prices = pd.DataFrame({'ESU2': near, 'ESZ2': far}, index=rng)

In [25]:
prices.tail()

,ESU2,ESZ2
2019-02-27,2922.649963,2924.399963
2019-02-28,2908.899963,2911.149963
2019-03-01,2916.899963,2918.649963
2019-03-04,2933.399963,2932.649963
2019-03-05,2913.399963,2911.149963


In [26]:
def get_roll_weights(start, expiry, items, roll_periods=5):
    # start : first date to compute weighting DataFrame
    # expiry : Series of ticker -> expiration dates
    # items : sequence of contract names

    dates = pd.date_range(start, expiry[-1], freq='B')
    weights = pd.DataFrame(np.zeros((len(dates), len(items))),
                        index=dates, columns=items)

    prev_date = weights.index[0]
    for i, (item, ex_date) in enumerate(expiry.iteritems()):
        if i < len(expiry) - 1:
            weights.ix[prev_date:ex_date - pd.offsets.BDay(), item] = 1
            roll_rng = pd.date_range(end=ex_date - pd.offsets.BDay(),
                                     periods=roll_periods + 1, freq='B')

            decay_weights = np.linspace(0, 1, roll_periods + 1)
            weights.ix[roll_rng, item] = 1 - decay_weights
            weights.ix[roll_rng, expiry.index[i + 1]] = decay_weights
        else:
            weights.ix[prev_date:, item] = 1

        prev_date = ex_date

    return weights

In [27]:
weights = get_roll_weights('6/1/2012', expiry, prices.columns)
weights.loc['2012-09-12':'2012-09-21']

C:\Users\LubichJD\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\LubichJD\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\LubichJD\Anaconda2\lib\site-packages\ipykernel\__main__.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\LubichJD\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: DeprecationWarning: 
.ix is deprec

,ESU2,ESZ2
2012-09-12,1.0,0.0
2012-09-13,1.0,0.0
2012-09-14,0.8,0.2
2012-09-17,0.6,0.4
2012-09-18,0.4,0.6
2012-09-19,0.2,0.8
2012-09-20,0.0,1.0
2012-09-21,0.0,1.0


In [28]:
rolled_returns = (prices.pct_change() * weights).sum(1)